In [1]:
import requests
import pandas as pd
r = requests.get('https://statsapi.web.nhl.com/api/v1/game/2019020156/feed/live')
data = r.json()

copyright = data['copyright']
game_id = data['gamePk']
api_link = data['link']
timestamp = data['metaData']['timeStamp']
gameData = data['gameData']
liveData = data['liveData']


In [130]:
gameData = data['gameData']
game = gameData['game']
datetime = gameData['datetime']
status = gameData['status']
players = gameData['players']
venue = gameData['venue']

gameStart = datetime['dateTime']
gameEnd = datetime['endDateTime']

progress = status['detailedState']

{'situationTimeRemaining': 0, 'situationTimeElapsed': 0, 'inSituation': False}


In [2]:
teams = gameData['teams']
homeTeam = teams['home']['triCode']
awayTeam = teams['away']['triCode']

def createAdvantages(d, team):
    start = []
    end = []
    skaterDifference = []
    teams = []
    times = []
    for item in d:
        print(item)
        start.append(item['start'])
        end.append(item['end'])
        skaterDifference.append(item['skaterDifference'])
        teams.append(team)
        times.append(item['end'] - item['start'])
    return d

gamePressures = liveData['shotPressure']['gamePressures']
print(liveData['shotPressure']['skaterAdvantages'])
d = createAdvantages(liveData['shotPressure']['skaterAdvantages']['homeTeam'], 'Home')
a = createAdvantages(liveData['shotPressure']['skaterAdvantages']['awayTeam'], 'Away')


{'homeTeam': [{'start': 0, 'end': 0, 'skaterDifference': 1}, {'start': 3526, 'end': 3646, 'skaterDifference': 1}, {'start': 3579, 'end': 3699, 'skaterDifference': 1}], 'awayTeam': [{'start': 3554, 'end': 3674, 'skaterDifference': 1}, {'start': 2299, 'end': 2419, 'skaterDifference': 1}, {'start': 3145, 'end': 3233, 'skaterDifference': 1}]}
{'start': 0, 'end': 0, 'skaterDifference': 1}
{'start': 3526, 'end': 3646, 'skaterDifference': 1}
{'start': 3579, 'end': 3699, 'skaterDifference': 1}
{'start': 3554, 'end': 3674, 'skaterDifference': 1}
{'start': 2299, 'end': 2419, 'skaterDifference': 1}
{'start': 3145, 'end': 3233, 'skaterDifference': 1}


In [34]:
def calculateField(ht, at, field, gst):
    hts = ht[gst]['shots']['count']
    htms = ht[gst]['missedShots']['count']
    htg = ht[gst]['goals']['count']
    
    ats = at[gst]['shots']['count']
    atms = at[gst]['missedShots']['count']
    atg = at[gst]['goals']['count']
    
    hcf = hts + htms + htg
    acf = ats + atms + atg

    if field == 'corsi':
        htbs = ht[gst]['blockedShots']['count']
        atbs = at[gst]['blockedShots']['count']
        hcf += atbs
        acf += htbs
    
    ht[gst][f'{field}For'] = hcf
    at[gst][f'{field}For'] = acf
    ht[gst][f'{field}Against'] = acf
    at[gst][f'{field}Against'] = hcf
    
    ht[gst][f'{field}'] = ht[gst][f'{field}For'] - ht[gst][f'{field}Against']
    at[gst][f'{field}'] = at[gst][f'{field}For'] - at[gst][f'{field}Against']
    
    try:
        ht[gst][f'{field}ForPercent'] = round(ht[gst][f'{field}For'] / (ht[gst][f'{field}For'] + ht[gst][f'{field}Against']), 2)
        at[gst][f'{field}ForPercent'] = round(at[gst][f'{field}For'] / (at[gst][f'{field}For'] + at[gst][f'{field}Against']), 2)
    except ZeroDivisionError:
        ht[gst][f'{field}ForPercent'] = 0.0
        ht[gst][f'{field}ForPercent'] = 0.0
    
    return ht, at





plays = liveData['plays']
allPlays = plays['allPlays']
scoringPlays = plays['scoringPlays']
penaltyPlays = plays['penaltyPlays']
allPlays = plays['allPlays']
playsByPeriod = plays['playsByPeriod']
currentPlay = plays['currentPlay']
firstPeriod, secondPeriod, thirdPeriod = playsByPeriod
overtime = None
shootout = None

if len(playsByPeriod) > 3: overtime = playsByPeriod[3]
if len(playsByPeriod) > 4: shootout = playsByPeriod[4]

shotEvents = {
    'Blocked Shot': 'blockedShots',
    'Shot': 'shots',
    'Missed Shot': 'missedShots',
    'Goal': 'goals'
}

gameStatusTypes = {
    'EVEN': 'even',
    'PPG': 'pp',
    'SHG': 'sh'
}

shots = {
    'home': {
        'total': {},
        'even': {},
        'pp': {},
        'sh': {}
    },
    'away': {
        'total': {},
        'even': {},
        'pp': {},
        'sh': {}
    }
}

for team in shots:
    for status in shots[team]:
        shots[team][status] = {
            'shots': {
                'count': 0,
                'coordinates': []
            },
            'blockedShots': {
                'count': 0,
                'coordinates': []
            },
            'missedShots': {
                'count': 0,
                'coordinates': []
            },
            'goals': {
                'count': 0,
                'coordinates': []
            }
        }

for play in allPlays:
    result = play['result']
    strength = 'even'
    if 'strength' in result: strength = gameStatusTypes[result['strength']['code']]
    event = result['event']
    coordinates = play['coordinates']
    if event in shotEvents:
        t = play['team']['triCode']
        e = shotEvents[event]
        if e == 'goals' or e == 'shots':
            if e == 'goals':
                print(result)
                if t == homeTeam:
                    shots['home']['total'][e]['count'] += 1
                    shots['home'][strength][e]['count'] += 1
#                     shots['home'][e]['coordinates'].append(coordinates)
                else:
                    shots['away']['total'][e]['count'] +=1             
                    shots['away'][strength][e]['count'] +=1        
#                     shots['home']['shots']['coordinates'].append(coordinates)
            if play['about']['periodType'] != 'SHOOTOUT':
                if t == homeTeam:
                    shots['home']['total']['shots']['count'] += 1
                    shots['home'][strength]['shots']['count'] += 1
#                     shots['home']['shots']['coordinates'].append(coordinates)
                else:
                    shots['away']['total']['shots']['count'] +=1
                    shots['away'][strength]['shots']['count'] +=1
#                     shots['home']['shots']['coordinates'].append(coordinates)
        else:
            if t == homeTeam:
                shots['home']['total'][e]['count'] += 1
                shots['home'][strength][e]['count'] += 1
#                 shots['home'][e]['coordinates'].append(coordinates)
            else:
                shots['away']['total'][e]['count'] +=1
                shots['away'][strength][e]['count'] +=1
#                 shots['home'][e]['coordinates'].append(coordinates)



def calculateStats(ht, at):
    for gameStatusType in ht:
        calculateField(ht, at, 'corsi', gameStatusType)
        calculateField(ht, at, 'fenwick', gameStatusType)
    
    return ht, at

calculateStats(shots['home'], shots['away'])



{'event': 'Goal', 'eventCode': 'TOR29', 'eventTypeId': 'GOAL', 'description': 'Kevin Labanc (4) Deflected, assists: Brent Burns (8), Tomas Hertl (7)', 'secondaryType': 'Deflected', 'strength': {'code': 'EVEN', 'name': 'Even'}, 'gameWinningGoal': False, 'emptyNet': False}
{'event': 'Goal', 'eventCode': 'TOR502', 'eventTypeId': 'GOAL', 'description': 'Jake Muzzin (2) Wrist Shot, assists: Jason Spezza (2), Ilya Mikheyev (5)', 'secondaryType': 'Wrist Shot', 'strength': {'code': 'EVEN', 'name': 'Even'}, 'gameWinningGoal': False, 'emptyNet': False}
{'event': 'Goal', 'eventCode': 'TOR548', 'eventTypeId': 'GOAL', 'description': 'Morgan Rielly (3) Wrist Shot, assists: William Nylander (5)', 'secondaryType': 'Wrist Shot', 'strength': {'code': 'EVEN', 'name': 'Even'}, 'gameWinningGoal': True, 'emptyNet': False}
{'event': 'Goal', 'eventCode': 'TOR710', 'eventTypeId': 'GOAL', 'description': 'Ilya Mikheyev (4) Wrist Shot, assists: Mitchell Marner (10)', 'secondaryType': 'Wrist Shot', 'strength': {'c

({'total': {'shots': {'count': 27, 'coordinates': []},
   'blockedShots': {'count': 11, 'coordinates': []},
   'missedShots': {'count': 16, 'coordinates': []},
   'goals': {'count': 4, 'coordinates': []},
   'corsiFor': 63,
   'corsiAgainst': 38,
   'corsi': 25,
   'corsiForPercent': 0.62,
   'fenwickFor': 47,
   'fenwickAgainst': 27,
   'fenwick': 20,
   'fenwickForPercent': 0.64},
  'even': {'shots': {'count': 27, 'coordinates': []},
   'blockedShots': {'count': 11, 'coordinates': []},
   'missedShots': {'count': 16, 'coordinates': []},
   'goals': {'count': 4, 'coordinates': []},
   'corsiFor': 63,
   'corsiAgainst': 38,
   'corsi': 25,
   'corsiForPercent': 0.62,
   'fenwickFor': 47,
   'fenwickAgainst': 27,
   'fenwick': 20,
   'fenwickForPercent': 0.64},
  'pp': {'shots': {'count': 0, 'coordinates': []},
   'blockedShots': {'count': 0, 'coordinates': []},
   'missedShots': {'count': 0, 'coordinates': []},
   'goals': {'count': 0, 'coordinates': []},
   'corsiFor': 0,
   'corsiAga